In [525]:
%run web_scrape_func_def.ipynb

imports loaded.

> file management func:

    get_content(directory)
    get_last_update(file_path)

> fighters_stats_scrape func:

    get_xml(url, headers = {'User-Agent':'Moz...}) # Return web page data in .xml format.
    get_links(url)
    get_fighter_data(url) # Retrive data as list
    get_current_date()

> matches_stats_web_scrape func:

    get_event_date(xml)
    get_event_data(event_link)
    get_later_date(date1, date2)


# Main( )

#### User variables:

In [534]:
output_loc = '../data/'

#### Update requirements variables:

In [535]:
files = get_content(directory=output_loc)
last_file_update = None

for file in files:
    if 'ufc_fighters_stats' in file:
        last_file_update = get_last_update(file_path = output_loc + file)

print('Last update: ' + last_file_update)

Last update: 08-04-2023


In [528]:
last_file_update = None # date for testing

#### Get only required links:

In [529]:
events = get_links(url='http://ufcstats.com/statistics/events/completed?page=all')
events.tail(3)

# filter this df to only add new matches
if last_file_update == None:
        events_for_update = events.event_url
else:
    df = events[events.date > last_file_update].reset_index(drop=True)
    events_for_update = df.event_url
    
print('Events required update: %d' %len(events_for_update))    

Events required update: 641


#### Extracting data to data frame:

In [530]:
data = []
column_names = ['result','fighter','opponent',
                'f_kd','o_kd','f_str','o_str','f_td','o_td','f_sub','o_sub',
                'weight_class','method','type','round','time','date','event_url'
               ]

for url in tqdm(events_for_update):
    data.extend(get_event_data(event_link=url))

print('Process time = %f sec' %(end_time - start_time))     
df = pd.DataFrame(data=data, columns=column_names)  
df.tail(3)

100%|██████████| 641/641 [04:33<00:00,  2.34it/s]

Process time = 30.493451 sec


,result,fighter,opponent,f_kd,o_kd,f_str,o_str,f_td,o_td,f_sub,o_sub,weight_class,method,type,round,time,date,event_url
6428,win,Frank Hamaker,Thaddeus Luster,0,0,2,0,1,0,3,0,Open Weight,SUB,Keylock,1,4:52,11-03-1994,http://ufcstats.com/event-details/a6a9ab5a824e...
6429,win,Johnny Rhodes,David Levicki,0,0,11,4,1,0,0,0,Open Weight,KO/TKO,Punches,1,12:13,11-03-1994,http://ufcstats.com/event-details/a6a9ab5a824e...
6430,win,Patrick Smith,Ray Wizard,0,0,1,1,0,0,1,0,Open Weight,SUB,Guillotine Choke,1,0:58,11-03-1994,http://ufcstats.com/event-details/a6a9ab5a824e...


#### Sort by date:

In [532]:
df['date'] = pd.to_datetime(df.date, format='%d-%m-%Y') 
df = df.sort_values(by='date', ascending=False).reset_index(drop=True)
df.head(3)

,result,fighter,opponent,f_kd,o_kd,f_str,o_str,f_td,o_td,f_sub,o_sub,weight_class,method,type,round,time,date,event_url
0,win,Israel Adesanya,Alex Pereira,1,0,41,49,0,0,0,0,Middleweight,KO/TKO,Punch,2,4:21,2023-04-08,http://ufcstats.com/event-details/3dc3022232b7...
1,win,Kevin Holland,Santiago Ponzinibbio,1,0,66,44,0,0,0,0,Welterweight,KO/TKO,Punch,3,3:16,2023-04-08,http://ufcstats.com/event-details/3dc3022232b7...
2,win,Christian Rodriguez,Raul Rosas Jr.,0,0,29,2,1,3,0,1,Bantamweight,U-DEC,,3,5:00,2023-04-08,http://ufcstats.com/event-details/3dc3022232b7...


#### Fighters list that require update:

In [547]:
if last_file_update != None:
    fighter_list = df[['fighter','event_url']][df.date > last_file_update]
    opponent_list = df[['opponent','event_url']][df.date > last_file_update].rename(columns={'opponent':'fighter'})
    full_list = pd.concat([fighter_list,opponent_list]).drop_duplicates(ignore_index=True)
    
    full_list.to_csv(output_loc + 'fighters_required_update.csv', index=0)
    full_list.tail(3)

#### Saving file:

In [533]:
if last_file_update != None:
    file = pd.read_csv(output_loc + 'ufc_matches_stats.csv')
    df = pd.concat([df, file]).drop_duplicates(ignore_index=True)
    #df['date'] = pd.to_datetime(df.date, format='%d-%m-%Y') 
    print(output_loc + 'ufc_matches_stats.csv updated.')
else:
    print(output_loc + 'ufc_matches_stats.csv saved.')

df.to_csv(output_loc + 'ufc_matches_stats.csv', index=0)
df.tail(3)

../data/ufc_matches_stats.csv saved.


,result,fighter,opponent,f_kd,o_kd,f_str,o_str,f_td,o_td,f_sub,o_sub,weight_class,method,type,round,time,date,event_url
6428,win,Johnny Rhodes,David Levicki,0,0,11,4,1,0,0,0,Open Weight,KO/TKO,Punches,1,12:13,1994-03-11,http://ufcstats.com/event-details/a6a9ab5a824e...
6429,win,Royce Gracie,Minoki Ichihara,0,0,2,3,1,0,2,0,Open Weight,SUB,Armbar,1,5:08,1994-03-11,http://ufcstats.com/event-details/a6a9ab5a824e...
6430,win,Patrick Smith,Ray Wizard,0,0,1,1,0,0,1,0,Open Weight,SUB,Guillotine Choke,1,0:58,1994-03-11,http://ufcstats.com/event-details/a6a9ab5a824e...


# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>